In [2]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import pydicom

In [3]:
from codes.utils import get_mask, bounded, min_max_normalize, hu_clip

In [4]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [5]:
x_map = {}
y_map = {}

In [6]:
# train_case_path = 'data/train/*_*'
# test_case_path = 'data/test/*_*'

train_cbct_path = 'raw/train/*_cbct'
train_ct_path = 'raw/train/*_ct'
test_cbct_path = 'raw/test/*_cbct'
test_ct_path = 'raw/test/*_ct'

In [7]:
train_cbct_path = sorted(glob.glob(train_cbct_path))
train_ct_path = sorted(glob.glob(train_ct_path))
test_cbct_path = sorted(glob.glob(test_cbct_path))
test_ct_path = sorted(glob.glob(test_ct_path))

In [8]:
train_cbct_path

['raw/train/004_cbct',
 'raw/train/006_cbct',
 'raw/train/007_cbct',
 'raw/train/012_cbct',
 'raw/train/014_cbct',
 'raw/train/017_cbct',
 'raw/train/018_cbct',
 'raw/train/019_cbct',
 'raw/train/021_cbct',
 'raw/train/025_cbct',
 'raw/train/027_cbct',
 'raw/train/028_cbct',
 'raw/train/029_cbct',
 'raw/train/030_cbct',
 'raw/train/033_cbct',
 'raw/train/039_cbct',
 'raw/train/040_cbct',
 'raw/train/042_cbct',
 'raw/train/044_cbct',
 'raw/train/045_cbct',
 'raw/train/046_cbct',
 'raw/train/047_cbct',
 'raw/train/050_cbct',
 'raw/train/051_cbct',
 'raw/train/052_cbct',
 'raw/train/053_cbct',
 'raw/train/054_cbct',
 'raw/train/055_cbct',
 'raw/train/057_cbct',
 'raw/train/058_cbct',
 'raw/train/059_cbct',
 'raw/train/060_cbct',
 'raw/train/061_cbct',
 'raw/train/062_cbct',
 'raw/train/064_cbct',
 'raw/train/065_cbct',
 'raw/train/066_cbct',
 'raw/train/067_cbct',
 'raw/train/078_cbct',
 'raw/train/080_cbct',
 'raw/train/083_cbct',
 'raw/train/084_cbct',
 'raw/train/089_cbct',
 'raw/train

In [20]:
for folder in train_cbct_path:
    
    ff = folder.split("/")[-1]
    p_dicom = os.path.join("data", "train", "{}_img".format(ff))
    p_mask = os.path.join("data", "train", "{}_mask".format(ff))
    p_air = os.path.join("data", "train", "{}_air".format(ff))
    p_bone = os.path.join("data", "train", "{}_bone".format(ff))
    
    for filename in os.listdir(folder):
        
        ff_name = filename.split(".")[0]
        
        dicom = pydicom.read_file(os.path.join(folder, filename))
        dicom = dicom.pixel_array.copy()

        img = np.zeros(dicom.shape, dtype=np.float32)
        mask = np.zeros(dicom.shape, dtype=np.float32)
        air = np.zeros(dicom.shape, dtype=np.float32)
        bone = np.zeros(dicom.shape, dtype=np.float32)       
        
        if dicom.max() > -500 and len(np.unique(dicom)) != 1:
            img = hu_clip(dicom, 500, -500, False)
            x_min = dicom.min()
            x_max = dicom.max()
            
            if x_min == x_max:
                img = np.zeros(dicom.shape, dtype=np.float32)
                
            else:
                mask = hu_clip(dicom, -257, -512, True)
                mask = get_mask(mask)

                img = min_max_normalize(img, 500, -500)
                img = img * mask

                # air mask
                upper = ((-256) - (-500))/(500-(-500))
                lower = ((-257) - (-500))/(500-(-500))
                air = hu_clip(img, upper, lower, True)

                # bone mask
                upper = ((256) - (-500))/(500-(-500))
                lower = ((255) - (-500))/(500-(-500))
                bone = hu_clip(img, upper, lower, True)

#         try:
#             os.mkdir(p_dicom)
#             os.mkdir(p_mask)
#             os.mkdir(p_air)
#             os.mkdir(p_bone)
#         except FileExistsError:
#             print("file exists")

#         visualize(
#             dicom = dicom,
#             mask = mask,
#             air = air,
#             bone = bone,
#         )
            
        np.save(os.path.join(p_dicom, "{}.npy".format(ff_name)), img)
        np.save(os.path.join(p_mask, "{}.npy".format(ff_name)), mask)
        np.save(os.path.join(p_air, "{}.npy".format(ff_name)), air)
        np.save(os.path.join(p_bone, "{}.npy".format(ff_name)), bone)

In [21]:
for folder in train_ct_path:
    
    ff = folder.split("/")[-1]
    p_dicom = os.path.join("data", "train", "{}_img".format(ff))
    p_mask = os.path.join("data", "train", "{}_mask".format(ff))
    p_air = os.path.join("data", "train", "{}_air".format(ff))
    p_bone = os.path.join("data", "train", "{}_bone".format(ff))
    
    for filename in os.listdir(folder):
        
        ff_name = filename.split(".")[0]
        
        dicom = pydicom.read_file(os.path.join(folder, filename))
        dicom = dicom.pixel_array.copy()

        img = np.zeros(dicom.shape, dtype=np.float32)
        mask = np.zeros(dicom.shape, dtype=np.float32)
        air = np.zeros(dicom.shape, dtype=np.float32)
        bone = np.zeros(dicom.shape, dtype=np.float32)       
        
        if dicom.max() > -500 and len(np.unique(dicom)) != 1:
            img = hu_clip(dicom, 500, -500, False)
            x_min = dicom.min()
            x_max = dicom.max()
            
            if x_min == x_max:
                img = np.zeros(dicom.shape, dtype=np.float32)
                
            else:
                mask = hu_clip(dicom, -257, -512, True)
                mask = get_mask(mask)

                img = min_max_normalize(img, 500, -500)
                img = img * mask

                # air mask
                upper = ((-256) - (-500))/(500-(-500))
                lower = ((-257) - (-500))/(500-(-500))
                air = hu_clip(img, upper, lower, True)

                # bone mask
                upper = ((256) - (-500))/(500-(-500))
                lower = ((255) - (-500))/(500-(-500))
                bone = hu_clip(img, upper, lower, True)
            
#         try:
#             os.mkdir(p_dicom)
#             os.mkdir(p_mask)
#             os.mkdir(p_air)
#             os.mkdir(p_bone)
#         except FileExistsError:
#             print("file exists")
            
        np.save(os.path.join(p_dicom, "{}.npy".format(ff_name)), img)
        np.save(os.path.join(p_mask, "{}.npy".format(ff_name)), mask)
        np.save(os.path.join(p_air, "{}.npy".format(ff_name)), air)
        np.save(os.path.join(p_bone, "{}.npy".format(ff_name)), bone)

In [22]:
for folder in test_cbct_path:
    
    ff = folder.split("/")[-1]
    p_dicom = os.path.join("data", "test", "{}_img".format(ff))
    p_mask = os.path.join("data", "test", "{}_mask".format(ff))
    p_air = os.path.join("data", "test", "{}_air".format(ff))
    p_bone = os.path.join("data", "test", "{}_bone".format(ff))
    
    for filename in os.listdir(folder):
        
        ff_name = filename.split(".")[0]
        
        dicom = pydicom.read_file(os.path.join(folder, filename))
        dicom = dicom.pixel_array.copy()

        img = np.zeros(dicom.shape, dtype=np.float32)
        mask = np.zeros(dicom.shape, dtype=np.float32)
        air = np.zeros(dicom.shape, dtype=np.float32)
        bone = np.zeros(dicom.shape, dtype=np.float32)       
        
        if dicom.max() > -500 and len(np.unique(dicom)) != 1:
            img = hu_clip(dicom, 500, -500, False)
            x_min = dicom.min()
            x_max = dicom.max()
            
            if x_min == x_max:
                img = np.zeros(dicom.shape, dtype=np.float32)
                
            else:
                mask = hu_clip(dicom, -257, -512, True)
                mask = get_mask(mask)

                img = min_max_normalize(img, 500, -500)
                img = img * mask

                # air mask
                upper = ((-256) - (-500))/(500-(-500))
                lower = ((-257) - (-500))/(500-(-500))
                air = hu_clip(img, upper, lower, True)

                # bone mask
                upper = ((256) - (-500))/(500-(-500))
                lower = ((255) - (-500))/(500-(-500))
                bone = hu_clip(img, upper, lower, True)

#         try:
#             os.mkdir(p_dicom)
#             os.mkdir(p_mask)
#             os.mkdir(p_air)
#             os.mkdir(p_bone)
#         except FileExistsError:
#             print("file exists")

#         visualize(
#             img = img,
#             mask = mask,
#             air = air,
#             bone = bone,
#         )
            
        np.save(os.path.join(p_dicom, "{}.npy".format(ff_name)), img)
        np.save(os.path.join(p_mask, "{}.npy".format(ff_name)), mask)
        np.save(os.path.join(p_air, "{}.npy".format(ff_name)), air)
        np.save(os.path.join(p_bone, "{}.npy".format(ff_name)), bone)

In [23]:
for folder in test_ct_path:
    
    ff = folder.split("/")[-1]
    p_dicom = os.path.join("data", "test", "{}_img".format(ff))
    p_mask = os.path.join("data", "test", "{}_mask".format(ff))
    p_air = os.path.join("data", "test", "{}_air".format(ff))
    p_bone = os.path.join("data", "test", "{}_bone".format(ff))
    
    for filename in os.listdir(folder):
        
        ff_name = filename.split(".")[0]
        
        dicom = pydicom.read_file(os.path.join(folder, filename))
        dicom = dicom.pixel_array.copy()

        img = np.zeros(dicom.shape, dtype=np.float32)
        mask = np.zeros(dicom.shape, dtype=np.float32)
        air = np.zeros(dicom.shape, dtype=np.float32)
        bone = np.zeros(dicom.shape, dtype=np.float32)       
        
        if dicom.max() > -500 and len(np.unique(dicom)) != 1:
            img = hu_clip(dicom, 500, -500, False)
            x_min = dicom.min()
            x_max = dicom.max()
            
            if x_min == x_max:
                img = np.zeros(dicom.shape, dtype=np.float32)
                
            else:
                mask = hu_clip(dicom, -257, -512, True)
                mask = get_mask(mask)

                img = min_max_normalize(img, 500, -500)
                img = img * mask

                # air mask
                upper = ((-256) - (-500))/(500-(-500))
                lower = ((-257) - (-500))/(500-(-500))
                air = hu_clip(img, upper, lower, True)

                # bone mask
                upper = ((256) - (-500))/(500-(-500))
                lower = ((255) - (-500))/(500-(-500))
                bone = hu_clip(img, upper, lower, True)
            
#         try:
#             os.mkdir(p_dicom)
#             os.mkdir(p_mask)
#             os.mkdir(p_air)
#             os.mkdir(p_bone)
#         except FileExistsError:
#             print("file exists")
            
        np.save(os.path.join(p_dicom, "{}.npy".format(ff_name)), img)
        np.save(os.path.join(p_mask, "{}.npy".format(ff_name)), mask)
        np.save(os.path.join(p_air, "{}.npy".format(ff_name)), air)
        np.save(os.path.join(p_bone, "{}.npy".format(ff_name)), bone)

In [ ]:
index = 21
hu_range = (-512, -257)

In [ ]:
cbct_slices = read_dicom(train_cbct_path[index])
ct_slices = read_dicom(train_ct_path[index])
region = valid_slices(cbct_slices, ct_slices)

# ditch first and last 3
cbcts = cbct_slices[region[0] + 3: region[1] - 3]
cts = ct_slices[region[0] + 3: region[1] - 3]

img = cbcts[0].pixel_array.copy()
img = hu_clip(img, -256, -257, True)
img = (img * 255).astype(np.uint8)

In [ ]:
np.unique(img)

In [ ]:
plt.hist(img.flatten(), color='c')

In [ ]:
cnts, hier = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
c = max(cnts, key = cv2.contourArea)

In [ ]:
tmp = np.stack((img, img, img), axis=-1)
cv2.drawContours(tmp, [c], 0, (0, 255, 0), 3)
plt.figure(0, figsize=(6,6))
plt.imshow(tmp)

In [ ]:
mask = np.zeros((img.shape[0], img.shape[1], 3))
cv2.drawContours(mask, [c], 0, (255, 255, 255), -1)
plt.figure(0, figsize=(6,6))
plt.imshow(mask)